In [ ]:
import sfacts as sf

Load the simulated metagenotype and filter

- positions by a minimum minor allele frequency
- samples by a minimum horizontal coverage (fraction of sites with counts)

In [ ]:
%%bash
sfacts filter_mgen \
    --min-minor-allele-freq 0.05 \
    --min-horizontal-cvrg 0.1 \
    --random-seed 0 \
    sim.mgen.nc sim.filt.mgen.nc

In [ ]:
mgen_raw = sf.data.Metagenotypes.load('sim.mgen.nc')
mgen_filt = sf.data.Metagenotypes.load('sim.filt.mgen.nc')
print(mgen_raw.sizes)
print(mgen_filt.sizes)

We can see that this only reduced the data size by 3 samples and no positions.
(Real data will often be filtered more than this.)

Plotting the metagenotypes to see the slightly reduced dimensions:

In [ ]:
sf.plot.plot_metagenotype(mgen_raw.to_world())
sf.plot.plot_metagenotype(mgen_filt.to_world())

Now the fun part: fitting the StrainFacts model to these data.

A number of hyperparameters are set to their default values for this model.
In addition, we explicitly fit 15 strains (5 more than the simulation actually had),
and we only use 250 randomly sampled SNP positions.
This results in a much faster fitting than if we had used all 5000 positions, although
we may have less information for differentiating strains.
Finally, we set a random seed for reproducibility.

In [ ]:
%%bash
sfacts fit \
    --verbose \
    --model-structure ssdd3_with_error \
    --num-strains 15 --num-positions 250 \
    --random-seed 0 \
    sim.filt.mgen.nc sim.filt.fit.world.nc

A model of this size on this dataset should fit relatively quickly (<10 minutes on my computer).

When run on the command-line, several pieces of information are printed to the screen,
thanks to the `--verbose` flag.

The result of this fit is a "world" with all of the parameter value point estimates.

Let's load this into Python and plot the inferred genotypes and relative abundances.

In [ ]:
fit = sf.data.World.load('sim.filt.fit.world.nc')

# Plot inferred relative abundances for each sample (the "community").
sf.plot.plot_community(
    fit,
    row_linkage_func=lambda w: w.genotypes.linkage("strain"),
    col_linkage_func=lambda w: w.metagenotypes.linkage("sample"),
)

# Plot the inferred genotypes of the 10 simulated strains.
sf.plot.plot_genotype(
    fit,
    row_linkage_func=lambda w: w.genotypes.linkage("strain"),
    col_linkage_func=lambda w: w.metagenotypes.linkage("position"),
)

Because we subsampled just 250 positions for this first fit,
we may want to refit the model, now conditioning on the
previously estimated relative abundances.

Here, we fit 1000 position chunks, which means refitting our
model 5 times.
(If we wanted to use a split-apply-combine strategy,
the `--num-positions` and `--block-number` options, combined
would allow us to limit our refitting to just a subset of positions.)

Several hyperparameters are set to the defaults for this model.
For this refitting we have explicitly set the regularization parameter, $\gamma^*$, to 1.0,
which removes the bias towards discrete genotypes.
The result is that our genotype estimates will be more "fuzzy",
incorporating more uncertainty.

In [ ]:
%%bash
sfacts fit_genotype \
    --verbose \
    --model-structure ssdd3_with_error \
    --hyperparameters gamma_hyper=1.0 \
    --block-size=250 \
    --block-number=0 \
    --random-seed=0 \
    sim.filt.fit.world.nc sim.filt.mgen.nc sim.filt.fit.refit-0.geno.nc

In [ ]:
%%bash
sfacts fit_genotype \
    --verbose \
    --model-structure ssdd3_with_error \
    --hyperparameters gamma_hyper=1.0 \
    --block-size=250 \
    --block-number=1 \
    --random-seed=0 \
    sim.filt.fit.world.nc sim.filt.mgen.nc sim.filt.fit.refit-1.geno.nc

In [ ]:
%%bash
sfacts concatenate_genotype_chunks \
            --metagenotype sim.filt.mgen.nc \
            --community sim.filt.fit.world.nc \
            --outpath sim.filt.fit.refit.world.nc \
            sim.filt.fit.refit-{0,1}.geno.nc

`concatenate_genotype_chunks` then recombines one or more genotype blocks refit in this step with the observed
metagenotype data and original community inference to build a new world file.

When we visualize these refit genotypes, we see that they look similar, but slightly "fuzzier"
than the original fit.

In [ ]:
refit = sf.data.World.load('sim.filt.fit.refit.world.nc')
refit = refit.sel(position=fit.position.astype(str))

# We can see that we get approximately the same genotypes, but more fuzzy this time.
sf.plot.plot_genotype(
    refit,
    row_linkage_func=lambda w: w.genotypes.linkage("strain"),
    col_linkage_func=lambda w: w.metagenotypes.linkage("position"),
)

Finally, we'll dump the relative abundance and genotype inferences out to TSV files.

Note that the genotypes for each strain in each position are encoded as a float between
where 0.0 means entirely reference and 1.0 means entirely alternative allele.

In [ ]:
%%bash
sfacts dump --tsv sim.filt.fit.refit.world.nc \
    --genotype sim.filt.fit.refit.geno.tsv \
    --community sim.filt.fit.refit.comm.tsv

head sim.filt.fit.refit.geno.tsv sim.filt.fit.refit.comm.tsv

In the next example, we'll compare this fit to the simulated ground-truth
in order to evaluate our performance.